<a href="https://colab.research.google.com/github/peterbabulik/RandomCircuitSampling/blob/main/Random_Circuit_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cirq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [3]:
import cirq
import numpy as np
from typing import List, Tuple
import random

def create_random_circuit(
    qubits: List[cirq.Qid],
    depth: int,
    two_qubit_gate_prob: float = 0.5
) -> cirq.Circuit:
    """
    Creates a random quantum circuit with the specified depth.

    Args:
        qubits: List of qubits to use in the circuit
        depth: Number of layers in the circuit
        two_qubit_gate_prob: Probability of adding a two-qubit gate vs single-qubit gate

    Returns:
        A random quantum circuit
    """
    circuit = cirq.Circuit()

    # Available single-qubit gates with proper instantiation
    def random_single_qubit_gate(qubit):
        gates = [
            cirq.X(qubit),
            cirq.Y(qubit),
            cirq.Z(qubit),
            cirq.H(qubit),
            cirq.T(qubit),
            cirq.Rx(rads=np.random.uniform(0, 2 * np.pi))(qubit),
            cirq.Ry(rads=np.random.uniform(0, 2 * np.pi))(qubit),
            cirq.Rz(rads=np.random.uniform(0, 2 * np.pi))(qubit)
        ]
        return random.choice(gates)

    for _ in range(depth):
        # First, apply random single-qubit gates to all qubits
        for qubit in qubits:
            circuit.append(random_single_qubit_gate(qubit))

        # Then, randomly apply two-qubit gates
        available_qubits = list(qubits)
        while len(available_qubits) >= 2 and random.random() < two_qubit_gate_prob:
            # Choose random pair of qubits
            q1 = random.choice(available_qubits)
            available_qubits.remove(q1)
            q2 = random.choice(available_qubits)
            available_qubits.remove(q2)

            # Apply random two-qubit gate
            if random.random() < 0.5:
                circuit.append(cirq.CNOT(q1, q2))
            else:
                circuit.append(cirq.ISWAP(q1, q2))

    return circuit

def perform_random_circuit_sampling(
    n_qubits: int,
    depth: int,
    n_circuits: int,
    shots_per_circuit: int = 1000
) -> List[Tuple[cirq.Circuit, List[str]]]:
    """
    Performs random circuit sampling experiment.

    Args:
        n_qubits: Number of qubits to use
        depth: Depth of each random circuit
        n_circuits: Number of different random circuits to generate
        shots_per_circuit: Number of measurements per circuit

    Returns:
        List of tuples containing (circuit, measurement_results)
    """
    # Create a linear array of qubits
    qubits = cirq.LineQubit.range(n_qubits)

    # Create simulator
    simulator = cirq.Simulator()

    results = []
    for i in range(n_circuits):
        # Generate random circuit
        circuit = create_random_circuit(qubits, depth)

        # Add measurements at the end
        circuit.append(cirq.measure(*qubits, key='m'))

        # Run the circuit multiple times
        result = simulator.run(circuit, repetitions=shots_per_circuit)

        # Convert results to strings of 0s and 1s
        measurements = [''.join(map(str, sample)) for sample in result.measurements['m']]

        results.append((circuit, measurements))

        # Print progress
        print(f"Completed circuit {i + 1}/{n_circuits}")

    return results

def analyze_results(results: List[Tuple[cirq.Circuit, List[str]]]):
    """
    Analyzes the results of random circuit sampling.

    Args:
        results: List of (circuit, measurements) tuples
    """
    for i, (circuit, measurements) in enumerate(results):
        print(f"\nCircuit {i + 1}:")
        print("Circuit diagram:")
        print(circuit)
        print(f"\nFirst 5 measurements (out of {len(measurements)}):")
        print(measurements[:5])

        # Calculate statistics
        unique_outputs = len(set(measurements))
        most_common = max(set(measurements), key=measurements.count)
        most_common_count = measurements.count(most_common)
        most_common_freq = most_common_count / len(measurements)

        print(f"\nStatistics:")
        print(f"Number of unique output bitstrings: {unique_outputs}")
        print(f"Most common bitstring: {most_common}")
        print(f"Frequency of most common bitstring: {most_common_freq:.3f}")
        print(f"Expected frequency for uniform distribution: {1/2**len(most_common):.3f}")

# Example usage
def main():
    # Parameters for the experiment
    N_QUBITS = 4
    CIRCUIT_DEPTH = 5
    N_CIRCUITS = 3
    SHOTS = 1000

    print("Starting Random Circuit Sampling experiment...")
    print(f"Parameters: {N_QUBITS} qubits, depth {CIRCUIT_DEPTH}, "
          f"{N_CIRCUITS} circuits, {SHOTS} shots per circuit")

    results = perform_random_circuit_sampling(
        n_qubits=N_QUBITS,
        depth=CIRCUIT_DEPTH,
        n_circuits=N_CIRCUITS,
        shots_per_circuit=SHOTS
    )

    analyze_results(results)

if __name__ == "__main__":
    main()

Starting Random Circuit Sampling experiment...
Parameters: 4 qubits, depth 5, 3 circuits, 1000 shots per circuit
Completed circuit 1/3
Completed circuit 2/3
Completed circuit 3/3

Circuit 1:
Circuit diagram:
                                                                  ┌──┐
0: ───Y───────────T───────────T───────Y───Rx(1.58π)─────────────────X────M('m')───
                                                                    │    │
1: ───Rx(1.25π)───Y───────────Y───────H────────────────iSwap───────Z┼────M────────
                                                       │            │    │
2: ───Rz(1.1π)────Ry(1.32π)───iSwap───Y───Ry(0.834π)───iSwap───────T┼────M────────
                              │                                     │    │
3: ───Z───────────Ry(1.81π)───iSwap───Z───Rx(0.17π)────Rx(1.2π)─────@────M────────
                                                                  └──┘

First 5 measurements (out of 1000):
['1011', '0011', '1011', '0010', '1011']

Statistics:
Nu